Filling data from `1996 - 2025` of S & P 500 data using `yfinance`.

In [ ]:
import yfinance as yf
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
import pandas as pd
from datetime import date

load_dotenv()

db_url = os.getenv('DB_URL')
db_conn = create_engine(db_url)

constituents_table = os.getenv('CONSTIUENTS_TABLE')
prices_table = os.getenv('PRICES_TABLE')

data_path = os.getenv('DATA_PATH')

Up to you to have an appropiate database url and table set up. There is:
1. `Date`: a `date` column.
2. `Close, High, Low, Open`: price columns; I would take 4 decimal points.
3. `Volum`: volume columns of type integer.

In [34]:
def get_ticker_data(ticker: str, start: str, end: str):
    df = yf.download(ticker, start=start, end=end, auto_adjust=True)
    df.columns = df.columns.map(lambda x: x[0] if isinstance(x, tuple) else x)
    df = df.reset_index()
    df.index.name = None
    df['ticker'] = ticker

    for col in ['Close', 'High', 'Low', 'Open']:
        df[col] = round(df[col], 4)

    df.rename(columns = {
        'Date': 'date',
        'Close': 'adj_close',
        'High': 'high',
        'Low': 'low',
        'Open': 'open',
        'Volume': 'volume',
        'ticker': 'ticker'
    }, inplace=True)

    return df

In [ ]:
# quick e.g of how this works
get_ticker_data('AAPL', '2023-01-01', '2025-01-01')

The below code changes a historical database of `S & P 500` constituents I found online (![]())

In [ ]:
s_and_p_500_historical = pd.read_csv(f'{data_path}/s_and_p_500.csv')

s_and_p_500_historical['tickers'] = s_and_p_500_historical['tickers'].str.split(',')
s_and_p_500_historical['date'] = pd.to_datetime(s_and_p_500_historical['date'])

s_and_p_500_historical = s_and_p_500_historical.sort_values('date')

active = set()
records = []

for i, row in s_and_p_500_historical.iterrows():
    current_date = row['date']
    current_tickers = set(row['tickers'])

    entered = current_tickers - active
    for ticker in entered:
        records.append({'ticker': ticker, 'start_date': current_date, 'end_date': None})

    left = active - current_tickers
    for ticker in left:
        for rec in reversed(records):
            if rec['ticker'] == ticker and rec['end_date'] is None:
                rec['end_date'] = current_date
                break

    active = current_tickers

s_and_p_timeline = pd.DataFrame(records)

In [16]:
s_and_p_timeline

,ticker,start_date,end_date
0,HM,1996-01-02,2001-12-17
1,DEC,1996-01-02,1998-06-12
2,UK,1996-01-02,2001-02-07
3,SIAL,1996-01-02,2015-11-18
4,CNP,1996-01-02,NaT
...,...,...,...
1225,WSM,2025-04-24,NaT
1226,DASH,2025-04-24,NaT
1227,EXE,2025-04-24,NaT
1228,COIN,2025-05-19,NaT


In [ ]:
s_and_p_timeline_with_industries = []

for i, row in s_and_p_timeline.iterrows():
    try:
        print(i)
        stock_info = yf.Ticker(row['ticker']).info
        new_row = row.copy()
        if 'sector' in stock_info:
            new_row['sector'] = stock_info['sector']
        else:
            new_row['sector'] = None

        s_and_p_timeline_with_industries.append(new_row)
    except Exception as e:
        print(e)
        new_row = row.copy()
        new_row['sector'] = None
        s_and_p_timeline_with_industries.append(new_row)

In [20]:
constituents = pd.DataFrame(s_and_p_timeline_with_industries)

In [29]:
constituents['sector'] = constituents['sector'].replace({
    'Consumer Defensive': 'Consumer Staples',
    'Consumer Cyclical': 'Consumer Discretionary',
    'Basic Materials': 'Materials',
    'Financial Services': 'Financials'
})

Write the constituents to a database table if you wish. The `psql` table I used is located in `/data/sql/constituents.sql`

In [32]:
constituents.to_sql('constituents', db_conn, if_exists='append', index=False)

230

Save constituents to a dump.

In [66]:
constituents.to_csv('../dump/constituents.csv', index=False)

Okay now we fill the actual prices. I worry that I will be blocked pretty quickly from pulling, so I'll add a try catch and loop. 

In [ ]:
aapl_idx = s_and_p_timeline[s_and_p_timeline['ticker'] == 'AAPL'].index[0]

In [ ]:
continue_fill = s_and_p_timeline.iloc[aapl_idx:]

In [ ]:
for _, row in continue_fill.iterrows():
    if row['start_date'] is pd.NaT:
        continue # something has gone wrong here... but constituents table should be fairly good.

    # actually we can only pull currently listed stocks

    ticker = row['ticker']
    start_date = pd.to_datetime(row['start_date']).strftime('%Y-%m-%d')
    end_date = row['end_date'] if row['end_date'] is not pd.NaT else date.today().strftime('%Y-%m-%d')

    ticker_price_data = get_ticker_data(ticker, start_date, end_date)
    ticker_price_data.to_sql(prices_table, db_conn, if_exists='append', index=False)

In [ ]:
# break up into dump files
total_rows = pd.read_sql("SELECT COUNT(*) FROM adjusted_historical", db_conn).iloc[0, 0]
num_parts = 5
chunk_size = total_rows // num_parts

for i in range(num_parts):
    offset = i * chunk_size
    limit = chunk_size if i < num_parts - 1 else total_rows - offset  # ensure we get all remaining rows

    query = f"""
        SELECT * FROM adjusted_historical
        ORDER BY date, ticker
        OFFSET {offset} LIMIT {limit}
    """

    df_chunk = pd.read_sql(query, db_conn)
    df_chunk.to_csv(f"../dump/adjusted_historical_part_{i+1}.csv", index=False)

Finally, add the `SPDR` sector ETFs to the prices database. We also add `SPY` as a fallback for any industries that do not have access to sector ETFs for a given period - we expect that this will have weaker betas and more noise thus likely causing more adverse selection.

In [47]:
sector_etfs = ['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU']

min_date = pd.read_sql(f'select min(date) from {prices_table}', db_conn)['min'][0].strftime('%Y-%m-%d')
max_date = pd.read_sql(f'select max(date) from {prices_table}', db_conn)['max'][0].strftime('%Y-%m-%d')

In [52]:
for s in sector_etfs:
    sector_data = get_ticker_data(s, min_date, max_date)
    print(f'******** {s} ********')
    print(f'{sector_data['date'].iloc[1]} to {sector_data['date'].iloc[-1]}')
    print('*********************')
    sector_data.to_sql(prices_table, db_conn, if_exists='append', index=False)

[*********************100%***********************]  1 of 1 completed


******** XLC ********
2018-06-20 00:00:00 to 2025-07-17 00:00:00
*********************


[*********************100%***********************]  1 of 1 completed


******** XLY ********
1998-12-23 00:00:00 to 2025-07-17 00:00:00
*********************


[*********************100%***********************]  1 of 1 completed


...

******** XLRE ********
2015-10-09 00:00:00 to 2025-07-17 00:00:00
*********************


[*********************100%***********************]  1 of 1 completed


******** XLK ********
1998-12-23 00:00:00 to 2025-07-17 00:00:00
*********************


[*********************100%***********************]  1 of 1 completed


******** XLU ********
1998-12-23 00:00:00 to 2025-07-17 00:00:00
*********************


In [55]:
spy = get_ticker_data('SPY', min_date, max_date)
spy.to_sql('adjusted_historical', db_conn, if_exists='append', index=False)

[*********************100%***********************]  1 of 1 completed


434

Now save ETFs as a dump.

In [ ]:
etfs = sector_etfs + ['SPY']
sql_etfs = ', '.join(f"'{e}'" for e in etfs)

etfs_data = pd.read_sql(
    f"""
    select * from adjusted_historical where ticker in ({sql_etfs})
    """,
    db_conn
)

etfs_data.to_csv('../dump/adjusted_historical_etfs.csv', index=False)